In [ ]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 100000 # Number of times the enviroment is ran
LOSS_CLIPPING = 0.2 # Approximated values stated in the original paper
ENTROPY_LOSS = 0.001 # Epochs to train the network (recommended between 3 and 30)
LR = 0.00025 # Learning rate 
NUM_ACTIONS = 4 # Number of possible action in the environment
NUM_STATES = 8 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
LAMBDA = 0.95 # Used in the original paper un the GAE
NORMALIZE = True # Whether to normalize GAE or not



# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER(lr=LR))

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction

    # Generalized Advantage Estimation implemented in the original paper
    def get_gaes(self, rewards, dones, values, next_values):
        # Dones are used to track when is the final step of an episode, so next values are no applied
        deltas = [r + GAMMA * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        
        # Convert list to array as .mean() and .std() are used later
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * GAMMA * LAMBDA * gaes[t + 1]

        target = gaes + values
        if NORMALIZE:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Get the advantage
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))

        # Stack info to unpack it in the custom loss
        y_true = actions 
        advantages = np.reshape(advantages, (2048))

        start_train = time.time()
        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, sample_weight=advantages, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, target, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        #print('Time: ', time.time()-start_train)
 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            LR *= 0.95
                            K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()  
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'LunarLander-v2'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score: -410.91		average: -410.91
Episode:     2		score: -348.11		average: -379.51
Episode:     3		score:  -99.32		average: -286.11
Episode:     4		score:  -48.62		average: -226.74
Episode:     5		score:  -76.96		average: -196.78
Episode:     6		score: -158.77		average: -190.45
Episode:     7		score:  -84.17		average: -175.27
Episode:     8		score: -159.45		average: -173.29
Episode:     9		score: -193.51		average: -175.54
Episode:    10		score: -323.99		average: -190.38
Episode:    11		score: -220.21		average: -193.09
Episode:    12		score: -337.56		average: -205.13
Episode:    13		score: -524.00		average: -229.66
Episode:    14		score: -340.30		average: -237.56
Episode:    15		score: -203.28		average: -235.28
Episode:    16		score: -216.58		average: -234.11
Episode:    17		score: -260.36		average: -235.65
Episode:    18		score:  -47.18		average: -225.18
Episode:    19		score: -132.18		avera

Episode:   161		score: -190.06		average: -392.34
Episode:   162		score: -296.01		average: -393.94
Episode:   163		score: -328.93		average: -395.76
Episode:   164		score: -612.25		average: -400.49
Episode:   165		score: -449.33		average: -403.41
Episode:   166		score: -688.28		average: -408.90
Episode:   167		score: -545.23		average: -412.74
Episode:   168		score: -188.86		average: -411.30
Episode:   169		score: -555.74		average: -412.72
Episode:   170		score: -325.59		average: -411.03
Episode:   171		score: -527.36		average: -413.30
Episode:   172		score: -615.82		average: -415.72
Episode:   173		score: -175.28		average: -415.02
Episode:   174		score: -226.54		average: -411.25
Episode:   175		score: -610.27		average: -413.75
Episode:   176		score: -218.61		average: -413.95
Episode:   177		score: -626.90		average: -416.92
Episode:   178		score: -314.27		average: -416.28
Episode:   179		score: -704.66		average: -418.28
Episode:   180		score: -242.47		average: -414.75
Episode:   181		scor

Episode:   329		score: -780.10		average: -570.08
Episode:   330		score: -454.89		average: -570.48
Episode:   331		score: -488.97		average: -571.12
Episode:   332		score: -934.54		average: -575.45
Episode:   333		score: -481.60		average: -576.98
Episode:   334		score: -485.16		average: -574.92
Episode:   335		score: -904.02		average: -580.08
Episode:   336		score: -512.07		average: -579.88
Episode:   337		score: -448.17		average: -577.33
Episode:   338		score: -371.25		average: -576.48
Episode:   339		score: -505.37		average: -575.69
Episode:   340		score: -525.23		average: -577.48
Episode:   341		score: -505.85		average: -577.38
Episode:   342		score: -453.08		average: -578.85
Episode:   343		score: -447.13		average: -578.85
Episode:   344		score: -499.17		average: -579.99
Episode:   345		score: -782.42		average: -582.44
Episode:   346		score: -314.27		average: -581.13
Episode:   347		score: -557.62		average: -581.86
Episode:   348		score: -1017.20		average: -587.06
Episode:   349		sco

Episode:   497		score: -608.71		average: -569.52
Episode:   498		score: -343.69		average: -568.88
Episode:   499		score: -791.26		average: -571.47
Episode:   500		score: -694.62		average: -573.77
Episode:   501		score: -511.01		average: -573.59
Episode:   502		score: -494.77		average: -573.64
Episode:   503		score: -832.73		average: -575.34
Episode:   504		score: -693.77		average: -577.43
Episode:   505		score: -834.60		average: -581.71
Episode:   506		score: -738.52		average: -581.97
Episode:   507		score: -716.55		average: -583.39
Episode:   508		score: -904.09		average: -588.07
Episode:   509		score: -642.86		average: -589.84
Episode:   510		score: -723.42		average: -589.90
Episode:   511		score: -494.94		average: -589.91
Episode:   512		score: -346.23		average: -587.00
Episode:   513		score: -751.63		average: -587.44
Episode:   514		score: -458.74		average: -588.99
Episode:   515		score: -377.18		average: -586.63
Episode:   516		score: -453.94		average: -586.88
Episode:   517		scor

Episode:   665		score: -341.68		average: -576.71
Episode:   666		score: -929.27		average: -578.33
Episode:   667		score: -441.96		average: -577.29
Episode:   668		score: -723.49		average: -581.21
Episode:   669		score: -540.56		average: -582.43
Episode:   670		score: -486.94		average: -582.77
Episode:   671		score: -863.48		average: -586.16
Episode:   672		score: -384.21		average: -582.90
Episode:   673		score: -575.19		average: -584.61
Episode:   674		score: -754.68		average: -585.01
Episode:   675		score: -694.21		average: -586.42
Episode:   676		score: -626.51		average: -589.41
Episode:   677		score: -648.81		average: -588.25
Episode:   678		score: -662.87		average: -590.48
Episode:   679		score: -334.69		average: -584.97
Episode:   680		score: -495.89		average: -585.80
Episode:   681		score: -336.51		average: -581.65
Episode:   682		score: -574.95		average: -582.41
Episode:   683		score: -541.29		average: -580.03
Episode:   684		score: -817.13		average: -583.25
Episode:   685		scor

...

Episode:  4374		score: -866.82		average: -576.85
Episode:  4375		score: -434.59		average: -573.28
Episode:  4376		score: -674.99		average: -575.77
Episode:  4377		score: -757.62		average: -577.85
Episode:  4378		score: -590.20		average: -570.95
Episode:  4379		score: -603.87		average: -571.93
Episode:  4380		score: -454.95		average: -572.53
Episode:  4381		score: -696.17		average: -573.88
Episode:  4382		score: -444.85		average: -573.73
Episode:  4383		score: -694.25		average: -573.50
Episode:  4384		score: -738.59		average: -572.96
Episode:  4385		score: -608.27		average: -570.62
Episode:  4386		score: -628.10		average: -572.13
Episode:  4387		score: -406.94		average: -571.80
Episode:  4388		score: -341.16		average: -570.39
Episode:  4389		score: -480.60		average: -569.26
Episode:  4390		score: -556.88		average: -569.68
Episode:  4391		score: -749.86		average: -569.52
Episode:  4392		score: -486.87		average: -570.38
Episode:  4393		score: -780.80		average: -572.29
Episode:  4394		scor

Episode:  4543		score: -533.31		average: -602.54
Episode:  4544		score: -551.71		average: -602.90
Episode:  4545		score: -435.00		average: -597.78
Episode:  4546		score: -748.72		average: -598.12
Episode:  4547		score: -354.99		average: -596.97
Episode:  4548		score: -515.88		average: -598.23
Episode:  4549		score: -449.63		average: -594.78
Episode:  4550		score: -586.52		average: -596.78
Episode:  4551		score: -948.58		average: -602.29
Episode:  4552		score: -538.56		average: -603.68
Episode:  4553		score: -500.05		average: -604.55
Episode:  4554		score: -687.20		average: -606.92
Episode:  4555		score: -474.41		average: -607.72
Episode:  4556		score: -431.44		average: -602.84
Episode:  4557		score: -436.93		average: -602.12
Episode:  4558		score: -776.01		average: -606.34
Episode:  4559		score: -842.78		average: -610.05
Episode:  4560		score: -767.62		average: -607.86
Episode:  4561		score: -695.10		average: -609.27
Episode:  4562		score: -499.98		average: -607.85
Episode:  4563		scor

Episode:  4710		score: -295.60		average: -586.05
Episode:  4711		score: -475.34		average: -585.09
Episode:  4712		score: -423.52		average: -579.75
Episode:  4713		score: -398.68		average: -577.97
Episode:  4714		score: -943.85		average: -581.27
Episode:  4715		score: -787.29		average: -583.91
Episode:  4716		score: -563.63		average: -584.72
Episode:  4717		score: -782.92		average: -587.72
Episode:  4718		score: -951.40		average: -587.50
Episode:  4719		score: -689.32		average: -585.78
Episode:  4720		score: -333.68		average: -583.21
Episode:  4721		score: -855.97		average: -584.11
Episode:  4722		score: -584.21		average: -584.67
Episode:  4723		score: -770.86		average: -588.50
Episode:  4724		score: -452.03		average: -582.46
Episode:  4725		score: -321.47		average: -578.08
Episode:  4726		score: -495.10		average: -578.27
Episode:  4727		score: -350.01		average: -578.30
Episode:  4728		score: -408.54		average: -575.14
Episode:  4729		score: -728.69		average: -577.56
Episode:  4730		scor

Episode:  4877		score: -574.06		average: -570.99
Episode:  4878		score: -330.03		average: -570.45
Episode:  4879		score: -443.04		average: -571.32
Episode:  4880		score: -563.90		average: -571.62
Episode:  4881		score: -398.30		average: -568.26
Episode:  4882		score: -431.50		average: -568.24
Episode:  4883		score: -875.37		average: -568.34
Episode:  4884		score: -582.79		average: -566.20
Episode:  4885		score: -381.14		average: -561.69
Episode:  4886		score: -570.72		average: -560.42
Episode:  4887		score: -436.50		average: -557.31
Episode:  4888		score: -835.77		average: -558.94
Episode:  4889		score: -882.23		average: -558.76
Episode:  4890		score: -960.57		average: -563.85
Episode:  4891		score: -574.01		average: -565.55
Episode:  4892		score: -468.09		average: -565.29
Episode:  4893		score: -493.56		average: -564.12
Episode:  4894		score: -388.94		average: -561.57
Episode:  4895		score: -499.68		average: -561.97
Episode:  4896		score: -504.34		average: -559.82
Episode:  4897		scor

Episode:  5045		score: -328.53		average: -576.39
Episode:  5046		score: -800.99		average: -581.68
Episode:  5047		score: -295.63		average: -579.65
Episode:  5048		score: -479.27		average: -573.36
Episode:  5049		score: -688.31		average: -574.79
Episode:  5050		score: -392.27		average: -573.74
Episode:  5051		score: -485.39		average: -573.59
Episode:  5052		score: -515.55		average: -574.18
Episode:  5053		score: -856.68		average: -579.03
Episode:  5054		score: -499.40		average: -575.21
Episode:  5055		score: -443.80		average: -571.97
Episode:  5056		score: -451.85		average: -567.71
Episode:  5057		score: -799.50		average: -570.78
Episode:  5058		score: -546.29		average: -569.96
Episode:  5059		score: -422.22		average: -566.92
Episode:  5060		score: -355.63		average: -566.03
Episode:  5061		score: -496.62		average: -563.94
Episode:  5062		score: -373.03		average: -563.18
Episode:  5063		score: -620.20		average: -564.34
Episode:  5064		score: -749.58		average: -566.27
Episode:  5065		scor